# k means ||

In [2]:
# import the python libraries to create/connect to a Spark Session
from pyspark.sql import SparkSession

# build a SparkSession 
#   connect to the master node on the port where the master node is listening (7077)
#   declare the app name 
#   configure the executor memory to 512 MB
#   either *connect* or *create* a new Spark Context
spark = SparkSession.builder \
    .master("spark://spark-master:7077")\
    .appName("prova iniziale")\
    .config("spark.executor.memory", "512m")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/27 14:20:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
# create a spark context
sc = spark.sparkContext

# print its status
sc

<SparkContext master=spark://spark-master:7077 appName=prova iniziale>

# Import data from sklearn



In [ ]:
#!pip install scikit-learn # to be run at the launch of "docker compose up"

In [5]:
import numpy as np
import pandas as pd
import random
import sklearn.datasets #va installato

from pyspark.sql.functions import col
from pyspark.sql.functions import mean
from pyspark.sql.functions import stddev
from pyspark.ml.feature import StandardScaler

In [29]:
# Hyperparameters
k = 10

In [7]:
# Import dataframe directly from sklearn
prova =  sklearn.datasets.fetch_kddcup99(percent10 = True, as_frame = True)
X = prova.data

# prova = sc.parallelize(sklearn.datasets.fetch_kddcup99(percent10 = True, as_frame = True))

In [8]:
# Show the dataframe (to be deleted for a bigger dataframe) (TO BE DELETED)
X

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,b'tcp',b'http',b'SF',310,1881,0,0,0,0,...,86,255,1.0,0.0,0.01,0.05,0.0,0.01,0.0,0.0
494017,0,b'tcp',b'http',b'SF',282,2286,0,0,0,0,...,6,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0
494018,0,b'tcp',b'http',b'SF',203,1200,0,0,0,0,...,16,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0
494019,0,b'tcp',b'http',b'SF',291,1200,0,0,0,0,...,26,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0


# Create a Spark dataframe
And then acquire important informations about the dataframe: # columns, # rows, # partitions, the schema, ...

In [9]:
# Create the spark dataframe with a smaller chunk of data (just to work easily)
X_smaller = X.iloc[random.sample(range(1, len(X.index)), 5_000)]
spark_X = spark.createDataFrame(X_smaller)

/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [10]:
# Lock the spark dataframe
spark_X.persist() # non serve?

23/08/27 14:20:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[duration: bigint, protocol_type: binary, service: binary, flag: binary, src_bytes: bigint, dst_bytes: bigint, land: bigint, wrong_fragment: bigint, urgent: bigint, hot: bigint, num_failed_logins: bigint, logged_in: bigint, num_compromised: bigint, root_shell: bigint, su_attempted: bigint, num_root: bigint, num_file_creations: bigint, num_shells: bigint, num_access_files: bigint, num_outbound_cmds: bigint, is_host_login: bigint, is_guest_login: bigint, count: bigint, srv_count: bigint, serror_rate: double, srv_serror_rate: double, rerror_rate: double, srv_rerror_rate: double, same_srv_rate: double, diff_srv_rate: double, srv_diff_host_rate: double, dst_host_count: bigint, dst_host_srv_count: bigint, dst_host_same_srv_rate: double, dst_host_diff_srv_rate: double, dst_host_same_src_port_rate: double, dst_host_srv_diff_host_rate: double, dst_host_serror_rate: double, dst_host_srv_serror_rate: double, dst_host_rerror_rate: double, dst_host_srv_rerror_rate: double]

In [11]:
# Check the number of partitions the DataFrame is divided into the three workers yet
spark_X.rdd.getNumPartitions()

2

In [12]:
# Define the number of rows and columns in the dataframe
n_rows = spark_X.count()

TO CHANGE THE COLUMN/ROWS NUMBER EVALUATION

In [13]:
# Delete the non-numerical values in an easy way
# clean_X = spark_X.select('*').drop('protocol_type', 'service', 'flag')
# clean_X.printSchema()

# Delete the non-numerical values in a reusable way
col_type = np.array(spark_X.dtypes)
types = col_type[:,1] 
colnames = col_type[:,0]
clean_X = spark_X.select([col(colnames[i]) for i in range(len(colnames)) if not types[i] == 'binary'])
# clean_X.printSchema()

In [14]:
# Normalize the numerical columns

i = 0
means = np.zeros(len(clean_X.columns))
devstd = np.zeros(len(clean_X.columns))
# Calculate mean and variance for each column
for column in clean_X.columns:
    means[i] = float(clean_X.select(mean(column)).head()[0])
    devstd[i] = float(clean_X.select(stddev(column)).head()[0])
    i = i+1

# norm_X = clean_X
means, devstd

(array([5.6077400e+01, 1.9860712e+03, 5.9295700e+02, 0.0000000e+00,
        7.8000000e-03, 0.0000000e+00, 4.3400000e-02, 0.0000000e+00,
        1.5160000e-01, 6.6000000e-03, 2.0000000e-04, 0.0000000e+00,
        1.4000000e-02, 4.0000000e-03, 8.0000000e-04, 8.0000000e-04,
        0.0000000e+00, 0.0000000e+00, 1.2000000e-03, 3.3276860e+02,
        2.9312240e+02, 1.7512000e-01, 1.7471200e-01, 5.7036000e-02,
        5.7702000e-02, 7.9374000e-01, 2.0272000e-02, 2.9096000e-02,
        2.3271320e+02, 1.8824100e+02, 7.5193000e-01, 3.2174000e-02,
        6.0325600e-01, 6.3140000e-03, 1.7502600e-01, 1.7451800e-01,
        5.8014000e-02, 5.7150000e-02]),
 array([8.31628381e+02, 7.27101314e+04, 3.92534794e+03, 0.00000000e+00,
        1.48807220e-01, 0.00000000e+00, 9.21350214e-01, 0.00000000e+00,
        3.58668606e-01, 1.00789244e-01, 1.41421356e-02, 0.00000000e+00,
        3.24692293e-01, 2.06863623e-01, 3.46352410e-02, 2.82757830e-02,
        0.00000000e+00, 0.00000000e+00, 3.46236878e-02, 2.13

TO BE NORMALIZED!!

# Step 0: Useful functions

In [15]:
# Define the function for the distance between the cluster centers and the pandas dataframe
def distance(xrow,broadC):
    '''Distance between a dataframe.row
    and the broadcasted list of centers
    in the form of dataframe.row:
    
    broadC = sc.broadcast(center_rows)
    xrow = clean_X.collect()[1]
    '''
    x = np.array(xrow)
    centers = broadC.value
    the_ds = np.zeros(len(centers))
    
    for c in range(len(centers)):
        c_array = np.array(centers[c])
        dist2 = np.linalg.norm(x - c_array)**2
        the_ds[c] = dist2
        
    return np.min(the_ds)

# Define the function for the logarithm of the cost phi
def phi(df,centers ):
    '''logarithm of the cost phi'''
    return np.log( df.rdd.map(lambda row: distance(row,centers)).reduce(lambda x,y: x + y) )

# Is this reasonable?? --> maybe we should cast it to int ?
def evaluate_l(log_phi, k):
    return k/log_phi

# Function to select a row based on its probability
def select_row(x):
    if x > np.random.uniform(low = 0, high = 1):
        return True
    else:
        return False

### This is NOT $\Phi$. This is $log(\Phi)$. Pay attention budeo.

# Step 1: Choose a random sample from the dataset
This is the required step to begin the algorithm (doesn't need to be parallelized, since it is a select task)

In [38]:
# Choose the first sample randomly: select the random row
random_n = [np.random.randint(0, n_rows)]

# Define a function that is able to select the single row in the dataframe
def getrows(df, rownums=None): # copied from the internet
    return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: x[0])
random_sample = getrows(clean_X, random_n).collect()


In [39]:
# Broadcasting the row over the workers
bCent = sc.broadcast(random_sample)

# To show the content of the broadcast:
# b.value

# Step 2: Initial cost
Then we evaluate the cost function (sum of the squares after the first selection)

In [40]:
# Initial cost
initial_cost = phi(clean_X, bCent)

## Step 2.5: evaluate number of iterations

After having evaluated $\log \phi$, we find a number of iterations which is of the same order of magnitude.

In [41]:
int(initial_cost)

30

In [42]:
n_iter = int(initial_cost)

# Step 3: For loop
Implement the for loop in order to evaluate probabilities and choose new centers



In [43]:
# Loop over the log(phi) steps
# for i in range(int(initial_cost))
phi_iter = initial_cost
l = evaluate_l(phi_iter, k)

#Add initial probabilities
#probabilities = clean_X.rdd.map(lambda row: np.exp(np.log(distance(row,bCent))-phi_iter)*l)

#clean_X_p = clean_X.withColumn('probabilities', col(probabilities)) 

#distance = (clean_X.select(columns)
 #                      .withColumn('initial_cost', sum((col(colname)-random_sample[0][colname])**2 for colname in colnames))
  #             )

In [44]:
l

0.3235301866565343

In [45]:
clean_X.count()

5000

Exception in thread "serve RDD 892" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:474)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [48]:
i = 0
while i < n_iter or len(bCent.value) < k:
    '''
    Nel ciclo for:
        - Evaluate for each row l * d()^2 / phi
        - Sample with that probability
        - Broadcast centers to nodes
        - Evaluate new cost
        
    Ricordiamoci che distance è già al quadrato e che phi è il logaritmo
    '''

    # Evaluate the probability and select the new rows
    #probabilities = clean_X.rdd.map(lambda row: np.exp(np.log(distance(row,bCent))-phi_iter)*l)
    new_rows = clean_X.rdd\
    .filter(lambda row: select_row(np.exp(np.log(distance(row,bCent))-phi_iter)*l)).collect()
    
    # Update the broadcast
    bCent = sc.broadcast(bCent.value + new_rows)
    
    # Evaluate new cost
    phi_iter =  phi(clean_X, bCent)
    
    i += 1

In [49]:
phi_iter

22.912637325607122

In [50]:
bCent.value

[Row(duration=0, src_bytes=520, dst_bytes=0, land=0, wrong_fragment=0, urgent=0, hot=0, num_failed_logins=0, logged_in=0, num_compromised=0, root_shell=0, su_attempted=0, num_root=0, num_file_creations=0, num_shells=0, num_access_files=0, num_outbound_cmds=0, is_host_login=0, is_guest_login=0, count=511, srv_count=511, serror_rate=0.0, srv_serror_rate=0.0, rerror_rate=0.0, srv_rerror_rate=0.0, same_srv_rate=1.0, diff_srv_rate=0.0, srv_diff_host_rate=0.0, dst_host_count=255, dst_host_srv_count=255, dst_host_same_srv_rate=1.0, dst_host_diff_srv_rate=0.0, dst_host_same_src_port_rate=1.0, dst_host_srv_diff_host_rate=0.0, dst_host_serror_rate=0.0, dst_host_srv_serror_rate=0.0, dst_host_rerror_rate=0.0, dst_host_srv_rerror_rate=0.0),
 Row(duration=5049, src_bytes=5133876, dst_bytes=0, land=0, wrong_fragment=0, urgent=0, hot=5, num_failed_logins=0, logged_in=1, num_compromised=0, root_shell=0, su_attempted=0, num_root=0, num_file_creations=0, num_shells=0, num_access_files=0, num_outbound_cmd

# Step 4: Select a subset of the possible centroids using k-means ++

Using kmeans ++:

Algorithm 1 k-means++(k) initialization.
1: C ← sample a point uniformly at random from X
2: while |C| < k do
2
3:
Sample x ∈ X with probability dφX(x,C)
(C)
4:
C ← C ∪ {

In [ ]:
'''
- Prendere la lista dei centroidi e farla diventare un distributed dataset
- Fare una NUOVA lista di centroidi 
- for i in range(k):
    pesca un singolo dato con probabilità d(x)/phi --> come fare?
    aggiungilo alla lista

'''

# TO DO:
    - Capire come scegliere l
    - Fare la normalizzazione dei dati (se la facciamo)
    - Capire se abbiamo ottimizzato nel modo giusto
    - Implementare tutto k means (optional)
    - Creare un file gemello con un dataframe visualizzabile e vedere se ha senso
    - Iniziare a fare i benchmark su cloud veneto
    - Grafici etc...

# Step 2: Initial cost (OLD VERSION)
Then we evaluate the cost function (sum of the squares after the first selection)

In [ ]:
# Take the name and type of the relevant columns
coltypes = spark_X[[item[0] for item in spark_X.dtypes if item[1].startswith('double')]].dtypes

In [ ]:
# And then make a list of the names
columns = list(col(coltypes[i][0]) for i in range(len(coltypes)))
colnames = list(coltypes[i][0] for i in range(len(coltypes)))

In [ ]:
# To check that the columns are correctly selected
spark_X.select(columns).printSchema()

In [ ]:
# Creating a new column with the initial calue of the error (to be updated throughout the program)
initial_cost = (spark_X.select(columns)
                       .withColumn('initial_cost', sum((col(colname)-random_sample[0][colname])**2 for colname in colnames))
               )

In [ ]:
# To check if the evaluation of the squared errors works (TO BE DELETED)
initial_cost.toPandas()


In [ ]:
# Evaluate the total error (that will define the number of for loops done in the search)

from pyspark.sql.functions import sum
# phi = initial_cost.select(sum(initial_cost.initial_cost)).show()

phi = initial_cost.select(sum(initial_cost.initial_cost)).head()[0]
print(phi)
# IS IT A GOOD VALUE?


## Step 3: updating loop

# Stop worker and master
Stop the running Spark context (sc) and Spark session (spark)

In [ ]:
# sc.stop()
# spark.stop()

Finally, use `docker compose down` to stop and clear all running containers.